In [ ]:
import sys
sys.path.append('../../')

from bohemian import *
from matrix_generators import *

In [ ]:
# g must be a function that takes a positive integer, num_matrices, as input
# and returns an array of dimension (num_matrices, n, n)
def g(num_matrices):
    return random_tridiagonal_matrix([-1, 1], 20, num_matrices = num_matrices, d = 0)

## Parameters

In [ ]:
num_matrices = 10**8       # Total number of matrices to compute eigenvalues of
batch_size = 10**5         # Number of matrices to compute eigenvalues of in a batch 
                           # (this will be the value provided to g)
matrices_per_file = 10**7  # Number of matrices in each file

use_single_precision = True  # Store computed eigenvalues in single precision when saved (i.e. np.compled64)
compress_data = False        # Save data in the compressed numpy format (np.save_compressed), will slow down saving
n_jobs = 4                   # Number of threads to run in parallel

symmetry_real = False        # Add symmetry across the real axis (i.e. if a+bi is an eigenvalue, 
                             # add a-bi as an eigenvalue)
symmetry_imag = True         # Add symmetry across the imaginary axis (i.e. if a+bi is an eigenvalue, 
                             # add -a+bi as an eigenvalue)
# When both symmetry_real and symmetry_imag are True, both symmetries will be used. (i.e. if a+bi
# is an eigenvalue, a-bi, -a+bi, and -a-bi are added)

cm = plt.cm.hsv_r                # Colormap for image
background_color = (0,0,0,255)   # Backgroud color (r,g,b,a), 8-bit int (transparent background is allowed)

real_min = -2     # Left limit of image
real_max = 2      # Right limit of image
imag_min = -2     # Bottom limit of image
imag_max = 2      # Top limit of image
height = 2001     # Number of pixels for height, width is automatically computed such that each pixel is square

# Locations of saved data/histogram/images
data_dir = 'Data'
histogram_file = 'Histogram_{}.npy'.format(height)
images_dir = 'Images'

## Compute Eigenvalues

In [ ]:
%%time

compute_eigenvalues(g,
                    num_matrices = num_matrices,
                    batch_size = batch_size,
                    matrices_per_file = matrices_per_file,
                    file_start_idx = 0,
                    use_single_precision = use_single_precision,
                    data_dir = data_dir,
                    compress_data = compress_data,
                    verbose = 1,
                    n_jobs = n_jobs)

## Compute Histogram

In [ ]:
%%time

compute_histogram(histogram_file = histogram_file,
                  data_dir = data_dir,
                  height = height,
                  compress_data = compress_data,
                  symmetry_real = symmetry_real,
                  symmetry_imag = symmetry_imag,
                  num_files = num_matrices//matrices_per_file,
                  axisrange = [[real_min, real_max], [imag_min, imag_max]],
                  verbose = 1,
                  n_jobs = n_jobs)

## Plot Eigenvalues

In [ ]:
%%time

# The raw integer count values in the histogram (at each pixel) are
# transformed by the function given in the histogram_map argument.
# log(count+1) typically works well.
plot_histogram(histogram_file = histogram_file,
               background_color = background_color,
               image_dir = images_dir,
               image_name = 'Image-1.png',
               cm = cm,
               histogram_map = lambda x: np.log(x+1))